In [1]:
from pathlib import Path

In [2]:
VIDPATH = Path("/sbtal/riksdag-video/")

In [3]:
_SWE_MODEL = "jimregan/wav2vec2-swedish-phonetic-waxholm"

In [4]:
from transformers import pipeline
pipe = pipeline(model=_SWE_MODEL, device=1)

/home/joregan/miniconda3/envs/hf_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(_SWE_MODEL)
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}


In [7]:
from pydub import AudioSegment

In [16]:
def convert_audio(audio_id):
    parameters=["-ac", "1", "-acodec", "pcm_s16le", "-ar", "16000"]
    video_path = VIDPATH / f"{audio_id}"
    video = AudioSegment.from_file(str(video_path), "mp4")
    outname = f"/tmp/{audio_id}.wav"
    video.export(outname, format="wav", parameters=parameters)
    return outname

In [9]:
import json

In [11]:
JSON_PATH = Path("/home/joregan/rd_phonetic")

In [ ]:
for video in VIDPATH.glob("*.mp4"):
    audio = video.stem
    OUTFILE = JSON_PATH / f"{audio}.json"
    # if not OUTFILE.exists():
    wavfile = convert_audio(video.name)
    output = pipe(wavfile, chunk_length_s=10, return_timestamps="word")
    with open(str(OUTFILE), "w") as jsonf:
        json.dump(output, jsonf)

In [ ]:
outputs = set([y for x, y in FILE_MAPPING.items()])
len(outputs)

In [ ]:
reverse = {}
with open("mappings.tsv", "w") as mappings:
    for api, audio in FILE_MAPPING.items():
        if audio in reverse:
            continue
        reverse[audio] = api
        mappings.write(f"{api}\t{audio}\n")
